This notebook describes a classifier for whether the Turkish “de/da” and “ki” suffixes should be
separated or not.

For example, in “Öğrenciler de geldi” it is separated, but in “Öğrencilerde gelişme
var.” it is not separated.

# Imports
*   Import necessary libraries



In [11]:
import numpy as np
import tensorflow
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Providing the Data

In [12]:
# Read the annotated file
with open("annotated_sentences_for_de.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

# Data pre-processing
sentences = []
labels = []

for line in lines:
    parts = line.strip().split()
    if len(parts) >= 2:
        sentences.append(parts[0])
        labels.append(parts[-1])


# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1

# Convert sentences and labels to sequences
X = tokenizer.texts_to_sequences(sentences)
X = pad_sequences(X)
y = np.array([1 if 'B-ERR' in label else 0 for label in labels])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Define and Compile the Neural Network

In [13]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=X.shape[1]))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the Neural Network

In [14]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
6441/6441 [==============================] - 61s 8ms/step - loss: 0.0410 - accuracy: 0.9836 - val_loss: 0.0367 - val_accuracy: 0.9833
Epoch 2/10
6441/6441 [==============================] - 50s 8ms/step - loss: 0.0239 - accuracy: 0.9870 - val_loss: 0.0362 - val_accuracy: 0.9832
Epoch 3/10
6441/6441 [==============================] - 61s 9ms/step - loss: 0.0231 - accuracy: 0.9870 - val_loss: 0.0523 - val_accuracy: 0.9819
Epoch 4/10
6441/6441 [==============================] - 52s 8ms/step - loss: 0.0229 - accuracy: 0.9872 - val_loss: 0.0384 - val_accuracy: 0.9834
Epoch 5/10
6441/6441 [==============================] - 50s 8ms/step - loss: 0.0227 - accuracy: 0.9871 - val_loss: 0.0470 - val_accuracy: 0.9834
Epoch 6/10
6441/6441 [==============================] - 48s 8ms/step - loss: 0.0226 - accuracy: 0.9872 - val_loss: 0.0483 - val_accuracy: 0.9835
Epoch 7/10
6441/6441 [==============================] - 50s 8ms/step - loss: 0.0226 - accuracy: 0.9872 - val_loss: 0.0394 - val_ac

# Save the Model

In [15]:
import pickle

# Specify the path where you want to save the model
model_save_path = "/content/custom_model/model2.h5"

# Save the model
model.save(model_save_path)

# Optionally, you can also save the tokenizer if you need it for later use
tokenizer_save_path = "/content/custom_model/tokenizer2.pickle"
with open(tokenizer_save_path, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Test the Model

In [20]:
from keras.models import load_model

# Load the model
loaded_model2 = load_model(model_save_path)

# Optionally, load the tokenizer
with open(tokenizer_save_path, 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)


# Load and predict on sentences from "test_sentences_de.txt"
test_sentences_path = "test_sentences_de.txt"

with open(test_sentences_path, 'r', encoding='utf-8') as file:
    test_sentences = file.readlines()

treshold = 3.5015692567696988e-09

# Predictions
for sentence_to_predict in test_sentences:
  print(f'\nInput Sentence: {sentence_to_predict}')
  X_new = tokenizer.texts_to_sequences([sentence_to_predict])
  X_new = pad_sequences(X_new, maxlen=X.shape[1])

  prediction = model.predict(X_new)[0][0]
  prediction_label = 'Error Detected! Separate it!' if prediction < treshold else 'Correct :)'

  print(f'Prediction: {prediction_label} (Probability: {prediction})')


Input Sentence: Evde yağ kalmamış.

1/1 [==============================] - 0s 21ms/step
Prediction: Correct :) (Probability: 5.127697022544453e-08)

Input Sentence: Bende gelmek istiyordum.

1/1 [==============================] - 0s 22ms/step
Prediction: Error Detected! Separate it! (Probability: 1.763664103648921e-09)

Input Sentence: İnsanın bazen çıkıp gezmeside gerekiyor.

1/1 [==============================] - 0s 24ms/step
Prediction: Error Detected! Separate it! (Probability: 2.192735326644879e-09)

Input Sentence: Anahtarları piknikte kaybetmiş.

1/1 [==============================] - 0s 22ms/step
Prediction: Correct :) (Probability: 4.3529911408768385e-07)

Input Sentence: Babam geldiğinde gidelim.

1/1 [==============================] - 0s 21ms/step
Prediction: Correct :) (Probability: 3.5015692567696988e-09)

Input Sentence: Sadece hastalandığımda arıyorsun.

1/1 [==============================] - 0s 22ms/step
Prediction: Error Detected! Separate it! (Probability: 1.54423340